# Lab5 B: Generate your QNN to hardware 


In Lab5 A, we trained a quantised network, and in this lab we will compile it to a hardware format to deploy it on our FPGAs.

In [1]:
import os
from pathlib import Path
import urllib.request
import tarfile
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from brevitas.nn import QuantConv2d, QuantLinear, QuantReLU 
import torch.nn as nn

In [2]:
root_path = Path(".")  # replace with your root path

In [3]:
class QuantMLPKWS_Dropout(nn.Module):
    def __init__(self, num_classes=12, hidden_dim=256, dropout_p=0.2, w_bit=3, a_bit=3):
        super().__init__()
        self.in_features = 1 * 10 * 49

        # Layer 1: 490 -> 256
        self.fc1 = QuantLinear(
            in_features=self.in_features,
            out_features=hidden_dim,
            weight_bit_width=w_bit,   # W3
            bias=True,
            return_quant_tensor=False
        )
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.act1 = QuantReLU(
            bit_width=a_bit,          # A3
            return_quant_tensor=False
        )
        self.drop1 = nn.Dropout(p=dropout_p)

        # Layer 2: 256 -> 256
        self.fc2 = QuantLinear(
            in_features=hidden_dim,
            out_features=hidden_dim,
            weight_bit_width=w_bit,
            bias=True,
            return_quant_tensor=False
        )
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.act2 = QuantReLU(
            bit_width=a_bit,
            return_quant_tensor=False
        )
        self.drop2 = nn.Dropout(p=dropout_p)

        # Layer 3: 256 -> 256
        self.fc3 = QuantLinear(
            in_features=hidden_dim,
            out_features=hidden_dim,
            weight_bit_width=w_bit,
            bias=True,
            return_quant_tensor=False
        )
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.act3 = QuantReLU(
            bit_width=a_bit,
            return_quant_tensor=False
        )
        self.drop3 = nn.Dropout(p=dropout_p)

        # Output layer: 256 -> num_classes
        self.fc_out = QuantLinear(
            in_features=hidden_dim,
            out_features=num_classes,
            weight_bit_width=w_bit,
            bias=True,
            return_quant_tensor=False
        )

    def forward(self, x):
        # x: (B, 1, 10, 49) -> flatten -> (B, 490)
        x = x.view(x.size(0), -1)

        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.act3(x)
        x = self.drop3(x)

        x = self.fc_out(x)
        return x

In [4]:
import torch
from pathlib import Path
from brevitas.export import export_qonnx 

model = QuantMLPKWS_Dropout(
    num_classes=12,
    hidden_dim=256,
    dropout_p=0.3,
    w_bit=3,
    a_bit=3
).cpu()

# --- 1. Load trained weights ---
weight_dir = root_path / "weights"
state_dict = torch.load(weight_dir / "mlpw2a2_model_weights.pth", map_location="cpu")
model.load_state_dict(state_dict)
model.eval()  # Always switch to eval mode before export

# --- 2. Prepare dummy input ---
# The dummy input shape must match the model’s expected input (B, 1, 10, 49)
dummy_input = torch.randn(1, 1, 10, 49)

# --- 3. Export to QONNX (for FINN / FPGA deployment) ---
export_path = str(root_path / "exports" / "kws_mlp_w3a3_qonnx.onnx")

with torch.no_grad():
    export_qonnx(
        model,
        args=dummy_input,  # sometimes use input_t=dummy_input depending on brevitas version
        export_path=export_path
    )

print("QONNX model successfully exported to:", export_path)


ModuleNotFoundError: No module named 'onnxscript'

In [ ]:
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType

ready_model_filename = model_dir + "/mlp-ready.onnx"
input_shape = (1, 600)

# create a dummy input tensor for the export
input_a = np.random.randint(0, 1, size=input_shape).astype(np.float32)
input_a = 2 * input_a - 1
scale = 1.0
input_t = torch.from_numpy(input_a * scale)

#Move to CPU before export
model_for_export.cpu()

# Export to ONNX
export_qonnx(
    model_for_export, export_path=ready_model_filename, input_t=input_t
)

# clean-up
qonnx_cleanup(ready_model_filename, out_file=ready_model_filename)

# Setting the input datatype explicitly because it doesn't get derived from the export function
model = ModelWrapper(ready_model_filename)
model.set_tensor_datatype(model.graph.input[0].name, DataType["BIPOLAR"])
model.save(ready_model_filename)

print("Model saved to %s" % ready_model_filename)